In [2]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Churn_Customer_Lifetime_Value_prediction/research'

In [3]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Churn_Customer_Lifetime_Value_prediction'

In [ ]:
import urllib.request as request

filename, headers = request.urlretrieve(
    url="https://docs.google.com/spreadsheets/d/1taus8Wx2s9xcavripC_TMuDNF2vVSVZ_/edit?usp=sharing&ouid=113748405694445668234&rtpof=true&sd=true",
    filename="sample_data__technical_assessment.xlsx"
)

In [8]:
import pandas as pd

df = pd.read_excel(
    'artifacts/data_ingestion/sample_data__technical_assessment.xlsx',
)

In [20]:
df.head()

,activity_month,account_id,brand_id,reg_date,ftd_date,qp_date,ben_login_id,tracker_id,player_reg_product,total_deposit,total_handle,total_ngr
0,2023-11-01,185196607,PA,2023-11-21,2023-11-21,NaT,BCUSA22L,7102592,BETTING,10.0,210.0,13.18
1,2023-11-01,154497623,AZ,2022-10-28,2022-10-28,2022-10-28,actionnetwork,7039239,BETTING,0.0,5.0,4.00
2,2023-11-01,153249011,MI,2022-08-18,2023-12-21,NaT,starsportsmx,7049874,CASINO,0.0,0.0,0.00
3,2023-11-01,184718245,OH,2023-11-13,2023-11-13,2023-11-13,actionnetwork,7106987,BETTING,10.0,210.0,138.98
4,2023-11-01,183573211,OH,2023-11-06,2023-11-06,2023-11-06,BCUSA22NP,7106983,BETTING,10.0,210.0,-320.22


In [ ]:
for id in dff['account_id'].unique():
    filtered_df = dff.query("account_id == @id")
    
    if len(filtered_df) > 1:
        print(filtered_df)
        print('________________________________________________________________________________________')

In [ ]:
df = dff.copy()
date_columns = ['activity_month', 'reg_date', 'ftd_date', 'qp_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [ ]:
df['months_active'] = ((df['activity_month'] - df['ftd_date']).dt.days / 30).fillna(0).astype(int)
df_agg = df.groupby(['account_id', 'activity_month']).agg({
        'months_active': 'max',
        'total_deposit': 'sum',
        'total_handle': 'sum',
        'total_ngr': 'sum',
        'brand_id': 'first',
        'ben_login_id': 'first',
        'player_reg_product': 'first',
        'reg_date': 'first'
    }).reset_index()
# df = df.drop(columns=['activity_month'])

In [ ]:
n = dff.isnull().sum()

n = n[n > 0]

In [ ]:
n.index.to_list()

In [ ]:
print(list(dff.columns))

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
categorical_features = dff.select_dtypes(include='object').columns.to_list()

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = pd.DataFrame(
    encoder.fit_transform(dff[categorical_features]),
    columns=encoder.get_feature_names_out(categorical_features),
    index=dff.index
)

df = pd.concat([dff, encoded], axis=1)
df.drop(categorical_features, axis=1, inplace=True)

In [21]:
df.head()

,activity_month,account_id,brand_id,reg_date,ftd_date,qp_date,ben_login_id,tracker_id,player_reg_product,total_deposit,total_handle,total_ngr
0,2023-11-01,185196607,PA,2023-11-21,2023-11-21,NaT,BCUSA22L,7102592,BETTING,10.0,210.0,13.18
1,2023-11-01,154497623,AZ,2022-10-28,2022-10-28,2022-10-28,actionnetwork,7039239,BETTING,0.0,5.0,4.00
2,2023-11-01,153249011,MI,2022-08-18,2023-12-21,NaT,starsportsmx,7049874,CASINO,0.0,0.0,0.00
3,2023-11-01,184718245,OH,2023-11-13,2023-11-13,2023-11-13,actionnetwork,7106987,BETTING,10.0,210.0,138.98
4,2023-11-01,183573211,OH,2023-11-06,2023-11-06,2023-11-06,BCUSA22NP,7106983,BETTING,10.0,210.0,-320.22


In [18]:
import numpy as np

numeric_features = df.select_dtypes(include=object).columns.to_list()
numeric_features

['brand_id', 'ben_login_id', 'player_reg_product']

In [24]:
df['player_reg_product'].unique()

array(['BETTING', 'CASINO', 'POKER', 'BINGO', 'RACEBOOK'], dtype=object)

In [5]:
df_test = pd.read_csv('artifacts/data_transformation/test.csv')
c = df_test.columns.to_list()

In [21]:
if 'churned' in c:
    print(True)
else:
    print(False)

False


In [9]:
df.isnull().sum()

activity_month            0
account_id                0
brand_id                  0
reg_date                  0
ftd_date               7262
qp_date               21237
ben_login_id              0
tracker_id                0
player_reg_product        0
total_deposit             0
total_handle            162
total_ngr                 0
dtype: int64